In [ ]:
!pip install opendatasets

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
import opendatasets as od
#5acab72028e81c5fbbddde9eb6202108
od.download("https://www.kaggle.com/datasets/xiaopengzhang12/lung-cancer-mri-images")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: olanle
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/xiaopengzhang12/lung-cancer-mri-images


100%|██████████| 522M/522M [00:01<00:00, 334MB/s]


In [ ]:
# Data paths (adjust if necessary)
data_dir = "//content/lung-cancer-mri-images/lung_cancer_MRI_dataset/train"
data_dir2 = "//content/lung-cancer-mri-images/lung_cancer_MRI_dataset/validate"
categories = ["cancer", "no_cancer"] #lung_aca", "lung_n", "lung_scc"]

# Image preprocessing parameters
img_size = (128, 128)  # Resize images to a consistent size
batch_size = 32

# Image data generation
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, img_size)
    image = image / 255.0  # Normalize pixel values
    return image

In [ ]:
image_data = []
labels = []
for category_index, category in enumerate(categories):
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        try:
            img_array = preprocess_image(os.path.join(path, img))
            image_data.append(img_array)
            labels.append(category_index)
        except Exception as e:
            print(f"Error processing image: {img}, Error: {e}")

# Convert to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

In [ ]:
image_data = []
labels = []
for category_index, category in enumerate(categories):
    # Process images from both directories in one loop
    for data_dir in [data_dir, data_dir2]:  # Iterate through both directories
        path = os.path.join(data_dir, category)
        for img in os.listdir(path):
            try:
                img_array = preprocess_image(os.path.join(path, img))
                image_data.append(img_array)
                labels.append(category_index)
            except Exception as e:
                print(f"Error processing image: {img}, Error: {e}")

# Convert to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

In [ ]:
# One-hot encode labels
labels = tf.keras.utils.to_categorical(labels)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, labels, test_size=0.2, random_state=42)

In [ ]:
# Build the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5), # Added dropout for regularization
    layers.Dense(2, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.1) # Reduced epochs and added validation_split

Epoch 1/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 11s 84ms/step - accuracy: 0.6798 - loss: 0.6076 - val_accuracy: 0.7948 - val_loss: 0.4072
Epoch 2/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7875 - loss: 0.4027 - val_accuracy: 0.7729 - val_loss: 0.3857
Epoch 3/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8078 - loss: 0.3508 - val_accuracy: 0.8035 - val_loss: 0.3523
Epoch 4/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8189 - loss: 0.3174 - val_accuracy: 0.8079 - val_loss: 0.3637
Epoch 5/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8153 - loss: 0.3125 - val_accuracy: 0.7817 - val_loss: 0.3653
Epoch 6/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8235 - loss: 0.3029 - val_accuracy: 0.8166 - val_loss: 0.3319
Epoch 7/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8434 - loss: 0.2826 - val_accuracy: 0.8341 - val_loss: 0.3033
Epoch 8/10
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8757 - loss: 0.2243 - val_accuracy: 0.8122 - 

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.8525 - loss: 0.3095
Test Loss: 0.2966
Test Accuracy: 0.8514


In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step


In [ ]:
# Classification report and confusion matrix
print(classification_report(y_true_classes, y_pred_classes))
print(confusion_matrix(y_true_classes, y_pred_classes))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88       372
           1       0.77      0.82      0.80       200

    accuracy                           0.85       572
   macro avg       0.83      0.85      0.84       572
weighted avg       0.85      0.85      0.85       572

[[322  50]
 [ 35 165]]


In [ ]:
model.save("lung_cancer_model.h5")

In [33]:
#Upload image
from google.colab import files
uploaded = files.upload()

#Preprocess the image
import cv2
import numpy as np

img_size = (128, 128)
file_name = list(uploaded.keys())[0]

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, img_size)
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

image = preprocess_image(file_name)

#Load model if not already loaded
from tensorflow.keras.models import load_model
model = load_model("lung_cancer_model.h5")

#Predict and show confidence
class_names = ["cancer", "no_cancer"]
prediction = model.predict(image)[0]  # Get first (and only) result

#Print each class with its confidence
for i, class_name in enumerate(class_names):
    confidence = prediction[i] * 100
    print(f"{class_name.upper()}: {confidence:.2f}%")

#Final result
predicted_class = class_names[np.argmax(prediction)]
print(f"\n🧠 Final Prediction: {predicted_class.upper()} (with {np.max(prediction)*100:.2f}% confidence)")

Saving Healthy lungs.png to Healthy lungs.png


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step
CANCER: 11.16%
NO_CANCER: 90.42%

🧠 Final Prediction: NO_CANCER (with 90.42% confidence)


SAVING MODEL TO GITHUB
